## 0. 학습 세팅

### 1) 메모리 정리

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 2) 수정된 코드 자동 업데이트

In [2]:
%load_ext autoreload
%autoreload 2
import foolbox as fb

## 1. Load library

In [3]:
import os
import time
import copy
import numpy as np
"""
import json
import glob

import numpy as np
from PIL import Image

from torchmetrics.aggregation import MeanMetric
"""
import argparse
import easydict

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
"""
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as Datasets
"""
import torchvision.transforms as T

from torch.utils.tensorboard import SummaryWriter
"""
import torch.utils.data as data
"""
from torch.utils.data import Dataset, DataLoader, random_split
from torchmetrics.functional.classification import accuracy

from src.models import vgg11_config, vgg13_config, vgg16_config
from src.models import VGG, get_vgg_layers
from src.engines import train, evaluate, get_predictions
from src.engines import plot_most_correct_wrong, epoch_time, normalize_image
from src.utils import load_checkpoint, save_checkpoint, save_transform, save_best_param

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
# GridSearch
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import random
import numpy as np
import pandas as pd

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [5]:
def GridSearch_loop(trial: Trial) -> float:
    p
    #==================================================================================================
    # 2. Variable Declaration
    #==================================================================================================
    # Jupyter 환경
    args = easydict.EasyDict({
            "title" : "Retina_ML_Grid(lr_tf)",
            "device" : "cuda",
            "root" : "data",
            "use_data" : "Retina_Some_binary", # Retina_Some_binary # Retina_Some_binary_GAN284 # Retina_Some_binary_GAN852 # Retina_student
            "batch_size" : 32, # !!!
            "num_workers" : 2,
            "epochs" : 100, # !!!
            #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            # "lr" : 7.5e-5, # !!!
            "lr" : trial.suggest_loguniform("lr", 1e-6, 5e-3), # !!!
            #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            "logs" : "logs",
            "checkpoints" : "checkpoints",
            "transform_dir" : "transform_infor",
            "resume" : False,
            "train_ratio" : 0.8,
            "val_ratio" : 0.2,
            "test_ratio" : 1.0,
            "output_dim" : 2
        })
    
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    print("lr : ", args.lr)
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

    #==================================================================================================
    # 3. Image Data Preprocessing
    # 1) 이미지 변환
    #==================================================================================================
    # Build Dataset
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    # transform 임의 선택(1 ~ 최대 개수) + 이미지 변환
    list_transforms = [T.RandomRotation(180), T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
                        T.RandomAutocontrast(), T.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 5)),
                        T.RandomAffine(180, shear=20), T.RandomPerspective(fill=[255,255,255]),
                        T.RandomGrayscale(p=0.5), T.RandomResizedCrop((256, 256))]
    rand_number_transform = random.randint(1, len(list_transforms)-1)
    rand_list_transforms = random.sample(list_transforms, rand_number_transform)

    train_transforms = T.Compose([
        T.Resize((256, 256)), # 이미지 크기 재조절
        *rand_list_transforms,
        T.ToTensor(),
        T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
    
    print("train_transforms : ", train_transforms)
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    """
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    train_transforms = T.Compose([
        T.Resize((256, 256)), # 이미지 크기 재조절
        T.RandomOrder([
            # T.RandomRotation(5), # 이미지 회전(5도 이하)
            # T.RandomHorizontalFlip(0.5), # 이미지 좌우 대칭(50% 확률)
    #         T.RandomRotation(180),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomAutocontrast(),
    #         T.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 5)),
    #         T.RandomAffine(180, shear=20),
    #         T.RandomPerspective(fill=[255,255,255]),
    #         T.RandomGrayscale(p=0.5),
    #         T.RandomResizedCrop((256, 256)),
        ]),
        T.ToTensor(),
        T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    """

    test_transforms = T.Compose([
        T.Resize((256, 256)), 
        T.ToTensor(),
        T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
    
    # transform 정보 저장
    save_transform(args.transform_dir, train_transforms, test_transforms, args.title)

    #==================================================================================================
    # 2) 이미지 데이터셋 불러오기 + 이미지 변환
    #==================================================================================================
    train_path = f'C:\\Users\\Bang\\JupyterProjects\\RetinaProject\\DeepLearningPytorchExample\\data\\{args.use_data}\\train' 
    test_path = f'C:\\Users\\Bang\\JupyterProjects\\RetinaProject\\DeepLearningPytorchExample\\data\\{args.use_data}\\test'

    train_dataset = torchvision.datasets.ImageFolder(
        train_path, 
        transform = train_transforms
    )
    test_dataset = torchvision.datasets.ImageFolder(
        test_path, 
        transform = test_transforms
    )

    train_dataset_classes = train_dataset.classes
    test_dataset_classes = test_dataset.classes

    print("len(train_dataset) : ", len(train_dataset))
    print("train_dataset_classes : ", train_dataset_classes)
    print("train_dataset.__getitem__(18) : ", train_dataset.__getitem__(18))

    print("len(test_dataset) : ", len(test_dataset))
    print("test_dataset_classes : ", test_dataset_classes)
    print("test_dataset.__getitem__(18) : ", test_dataset.__getitem__(18))
    
    #==================================================================================================
    # 3) 훈련, 검증 데이터 분할 + 검증 이미지 재변환
    #==================================================================================================
    all_train_dataset_size = len(train_dataset)
    train_dataset_size = int(all_train_dataset_size * args.train_ratio)
    valid_dataset_size = all_train_dataset_size - train_dataset_size

    splited_train_dataset, splited_valid_dataset = random_split(train_dataset, [train_dataset_size, valid_dataset_size]) # 훈련 데이터셋, 검증 데이터셋 크기 결정

    splited_valid_dataset = copy.deepcopy(splited_valid_dataset)
    splited_valid_dataset.dataset.transform = test_transforms

    #==================================================================================================
    # 4) 데이터 로드 to 메모리
    #==================================================================================================
    train_loader = DataLoader(splited_train_dataset,
                                shuffle = True,
                                batch_size = args.batch_size, 
                                num_workers=args.num_workers
                            )
    val_loader = DataLoader(splited_valid_dataset,
                                batch_size = args.batch_size,
                                num_workers=args.num_workers
                            )
    test_loader = DataLoader(test_dataset,
                                batch_size = args.batch_size,
                                num_workers=args.num_workers
                            )

    print(f"train_dataset 개수 : {len(splited_train_dataset)}, " + f"train_loader 개수 : {len(train_loader)}") # train_data 개수, train_loader batch set 개수
    print(f"val_dataset 개수 : {len(splited_valid_dataset)}, " + f"val_loader 개수 : {len(val_loader)}") # val_data 개수, val_loader batch set 개수
    print(f"test_dataset 개수 : {len(test_dataset)}, " + f"test_loader 개수 : {len(test_loader)}") # test_data 개수, test_loader batch set 개수

    #==================================================================================================
    # 5) 시각화
    #==================================================================================================
    train_loader_iter = iter(train_loader)
    train_loader_images, train_loader_labels = train_loader_iter.next()

    test_loader_iter = iter(test_loader)
    test_loader_images, test_loader_labels = test_loader_iter.next()

    def imshow(img):
        np_img = img.numpy()
        plt.imshow(np.transpose(np_img, (1, 2, 0)))

        """
        print("np_img.shape : ", np_img.shape)
        print("(np.transpose(np_img, (1, 2, 0))).shape : ", (np.transpose(np_img, (1, 2, 0))).shape)
        """

    print("train_loader_labels : ", train_loader_labels)
    print("".join("%5s "%train_dataset_classes[train_loader_labels[j]] for j in range(32)))
    print("train_loader_images.shape : ", train_loader_images.shape)
    imshow(torchvision.utils.make_grid(train_loader_images, nrow=6))
    print("torchvision.utils.make_grid(train_loader_images).shape : ", torchvision.utils.make_grid(train_loader_images).shape)

    print("test_loader_labels : ", test_loader_labels)
    print("".join("%5s "%test_dataset_classes[test_loader_labels[j]] for j in range(32)))
    print("test_loader_images.shape : ", test_loader_images.shape)
    imshow(torchvision.utils.make_grid(test_loader_images, nrow=6))
    print("torchvision.utils.make_grid(test_loader_images).shape : ", torchvision.utils.make_grid(test_loader_images).shape)
    
    #==================================================================================================
    # 4. Model Define
    # 1) 모델 정의
    #==================================================================================================
    # Build model
    vgg11_layers = get_vgg_layers(vgg11_config, batch_norm = True)
    model = VGG(vgg11_layers, args.output_dim)
    model = model.to(args.device)

    #==================================================================================================
    # 2) 옵티마이저 + 손실함수 + 스케쥴러 + 메트릭 함수 정의
    #==================================================================================================
    # Build optimizer 
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # Build scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs * len(train_loader))

    # Build loss function
    loss_fn = nn.CrossEntropyLoss()

    # Build metric function
    """
    # 정확도 측정 함수
    def calculate_accuracy(y_pred, y): # ???
        top_pred = y_pred.argmax(1, keepdim = True)
        # print("top_pred : ", top_pred)
        correct = top_pred.eq(y.view_as(top_pred)).sum()
        # print("correct : ", correct)
        acc = correct.float() / y.shape[0]
        # print("acc : ", acc)
        return acc
    """
    metric_fn = accuracy

    #==================================================================================================
    # 3) logger 정의
    #==================================================================================================
    # Build logger
    train_logger = SummaryWriter(f'{args.logs}/train/{args.title}')
    val_logger = SummaryWriter(f'{args.logs}/val/{args.title}')
    test_logger = SummaryWriter(f'{args.logs}/test/{args.title}')

    #==================================================================================================
    # 5. Model Train
    # 1) Load model epoch
    #==================================================================================================
    # Load model
    start_epoch = 0
    if args.resume:
        start_epoch = load_checkpoint(args.checkpoints, args.title, model, optimizer)

    #==================================================================================================
    # 2) Train model
    #==================================================================================================
    best_valid_loss = float('inf')
    for epoch in range(start_epoch, args.epochs):
        """
        # start timer
        start_time = time.time() # 확인용 코드
        """
        # 모델 학습 소요시간
        start_time = time.monotonic()

        # train one epoch + evaluate one epoch
        train_summary = train(train_loader, model, optimizer, scheduler, loss_fn, metric_fn, args.device)
        val_summary = evaluate(val_loader, model, loss_fn, metric_fn, args.device)

        # write log
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        train_logger.add_scalar(f'Loss_{args.lr}', train_summary['loss'], epoch + 1)
        train_logger.add_scalar(f'Accuracy_{args.lr}', train_summary['metric'], epoch + 1)
        val_logger.add_scalar(f'Loss_{args.lr}', val_summary['loss'], epoch + 1)
        val_logger.add_scalar(f'Accuracy_{args.lr}', val_summary['metric'], epoch + 1)
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        """
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        train_logger.add_scalar('Loss', train_summary['loss'], epoch + 1)
        train_logger.add_scalar('Accuracy', train_summary['metric'], epoch + 1)
        val_logger.add_scalar('Loss', val_summary['loss'], epoch + 1)
        val_logger.add_scalar('Accuracy', val_summary['metric'], epoch + 1)
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        """

        # 최적 loss인 model 저장
        if val_summary['loss'] < best_valid_loss:
            best_valid_loss = val_summary['loss']
            torch.save(model.state_dict(), f'C:\\Users\\Bang\\JupyterProjects\\RetinaProject\\DeepLearningPytorchExample\\best_valid_model\\{args.title}_.pt')
            #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            save_best_param(args.transform_dir, train_transforms, test_transforms, args.lr, args.title)
            #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

        """
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        # save model
        save_checkpoint(args.checkpoints, args.title, model, optimizer, epoch + 1)
        #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
        """

        # 모델 학습 소요시간
        end_time = time.monotonic()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Print log
        """
        print((
            f'[Epoch {epoch+1}] '
            + f'{epoch + 1}epoch time {end_time - start_time:.02f}, '
            + f'Train Loss {train_summary["loss"]:.04f}, '
            + f'Train Accuracy {train_summary["metric"]:.04f}, '
            + f'Test Loss {val_summary["loss"]:.04f}, '
            + f'Test Accuracy {val_summary["metric"]:.04f}'
        ))
        """
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\t Train Loss: {train_summary["loss"]:.3f} | Train Acc: {train_summary["metric"]:.2f}%')
        print(f'\t Valid Loss: {val_summary["loss"]:.3f} | Valid Acc: {val_summary["metric"]:.2f}%')
        print(f'\t scheduled_lr : {scheduler.get_last_lr()[0]}')

    #==================================================================================================
    # 3) return loss
    #==================================================================================================
    return val_summary["loss"]

In [6]:
# GridSearch loop
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="Retina_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(GridSearch_loop, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

[I 2022-11-28 18:52:45,218] A new study created in memory with name: Retina_opt
C:\Users\Bang\AppData\Local\Temp\ipykernel_14404\1112370813.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "lr" : trial.suggest_loguniform("lr", 1e-6, 5e-3), # !!!


lr :  2.428916946974887e-05
train_transforms :  Compose(
    Resize(size=(256, 256), interpolation=bilinear)
    RandomAutocontrast(p=0.5)
    RandomPerspective(p=0.5)
    RandomHorizontalFlip(p=0.5)
    RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomRotation(degrees=[-180.0, 180.0], interpolation=nearest, expand=False, fill=0)
    RandomAffine(degrees=[-180.0, 180.0], shear=[-20.0, 20.0])
    RandomGrayscale(p=0.5)
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 5))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
len(train_dataset) :  4532
len(test_dataset) :  1132
train_dataset 개수 : 3625, train_loader 개수 : 114
val_dataset 개수 : 907, val_loader 개수 : 29
test_dataset 개수 : 1132, test_loader 개수 : 36
Epoch: 01 | Epoch Time: 0m 42s
	 Train Loss: 0.747 | Train Acc: 0.50%
	 Valid Loss: 0.720 | Valid Acc: 0.50%
	 scheduled_lr : 2.4283176850298953e-05
Epoch: 02 | Epoch Time: 0m 40s
	 Train Loss:

Epoch: 51 | Epoch Time: 0m 39s
	 Train Loss: 0.691 | Train Acc: 0.53%
	 Valid Loss: 0.688 | Valid Acc: 0.56%
	 scheduled_lr : 1.1763114109663376e-05
Epoch: 52 | Epoch Time: 0m 39s
	 Train Loss: 0.691 | Train Acc: 0.53%
	 Valid Loss: 0.689 | Valid Acc: 0.57%
	 scheduled_lr : 1.1382019949903898e-05
Epoch: 53 | Epoch Time: 0m 39s
	 Train Loss: 0.690 | Train Acc: 0.51%
	 Valid Loss: 0.687 | Valid Acc: 0.55%
	 scheduled_lr : 1.1001678349521632e-05
Epoch: 54 | Epoch Time: 0m 39s
	 Train Loss: 0.691 | Train Acc: 0.52%
	 Valid Loss: 0.689 | Valid Acc: 0.57%
	 scheduled_lr : 1.0622464659756943e-05
Epoch: 55 | Epoch Time: 0m 39s
	 Train Loss: 0.690 | Train Acc: 0.52%
	 Valid Loss: 0.686 | Valid Acc: 0.56%
	 scheduled_lr : 1.02447531187386e-05
Epoch: 56 | Epoch Time: 0m 39s
	 Train Loss: 0.689 | Train Acc: 0.53%
	 Valid Loss: 0.685 | Valid Acc: 0.59%
	 scheduled_lr : 9.868916482155945e-06
Epoch: 57 | Epoch Time: 0m 39s
	 Train Loss: 0.689 | Train Acc: 0.54%
	 Valid Loss: 0.686 | Valid Acc: 0.57%


[I 2022-11-28 19:59:55,175] Trial 0 finished with value: 0.6859443187713623 and parameters: {'lr': 2.428916946974887e-05}. Best is trial 0 with value: 0.6859443187713623.


Epoch: 100 | Epoch Time: 0m 39s
	 Train Loss: 0.684 | Train Acc: 0.55%
	 Valid Loss: 0.686 | Valid Acc: 0.58%
	 scheduled_lr : 0.0
lr :  0.0032859708169642424
train_transforms :  Compose(
    Resize(size=(256, 256), interpolation=bilinear)
    RandomAffine(degrees=[-180.0, 180.0], shear=[-20.0, 20.0])
    RandomPerspective(p=0.5)
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 5))
    RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomVerticalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
len(train_dataset) :  4532
len(test_dataset) :  1132
train_dataset 개수 : 3625, train_loader 개수 : 114
val_dataset 개수 : 907, val_loader 개수 : 29
test_dataset 개수 : 1132, test_loader 개수 : 36
Epoch: 01 | Epoch Time: 0m 40s
	 Train Loss: 27.299 | Train Acc: 0.49%
	 Valid Loss: 0.696 | Valid Acc: 0.52%
	 scheduled_lr : 0.00328516010284517
Epoch: 02 | Epoch Time: 0m 39s
	 Train Loss: 0.732 | Train Acc:

...

[I 2022-11-29 04:50:33,375] Trial 8 finished with value: 0.6924028992652893 and parameters: {'lr': 0.00016730402817820238}. Best is trial 6 with value: 0.6724250912666321.


Epoch: 100 | Epoch Time: 0m 39s
	 Train Loss: 0.691 | Train Acc: 0.50%
	 Valid Loss: 0.692 | Valid Acc: 0.48%
	 scheduled_lr : 0.0
lr :  0.0004160439645256605
train_transforms :  Compose(
    Resize(size=(256, 256), interpolation=bilinear)
    RandomVerticalFlip(p=0.5)
    RandomGrayscale(p=0.5)
    RandomAutocontrast(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
len(train_dataset) :  4532
len(test_dataset) :  1132
train_dataset 개수 : 3625, train_loader 개수 : 114
val_dataset 개수 : 907, val_loader 개수 : 29
test_dataset 개수 : 1132, test_loader 개수 : 36
Epoch: 01 | Epoch Time: 0m 40s
	 Train Loss: 1.175 | Train Acc: 0.52%
	 Valid Loss: 0.697 | Valid Acc: 0.48%
	 scheduled_lr : 0.0004159413182348131
Epoch: 02 | Epoch Time: 0m 40s
	 Train Loss: 0.712 | Train Acc: 0.50%
	 Valid Loss: 0.695 | Valid Acc: 0.50%
	 scheduled_lr : 0.00041563348066176673
Epoch: 03 | Epoch Time: 0m 40s
	 Train Loss: 0.702 | Train Acc: 0.49%
	 Valid Loss: 0.693 | Valid Acc: 0.

Epoch: 53 | Epoch Time: 0m 39s
	 Train Loss: 0.693 | Train Acc: 0.49%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.00018844538437890192
Epoch: 54 | Epoch Time: 0m 39s
	 Train Loss: 0.693 | Train Acc: 0.49%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.00018194991457337413
Epoch: 55 | Epoch Time: 0m 39s
	 Train Loss: 0.693 | Train Acc: 0.50%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.0001754801747509366
Epoch: 56 | Epoch Time: 0m 39s
	 Train Loss: 0.693 | Train Acc: 0.48%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.00016904254976369355
Epoch: 57 | Epoch Time: 0m 38s
	 Train Loss: 0.693 | Train Acc: 0.50%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.00016264339277028322
Epoch: 58 | Epoch Time: 0m 38s
	 Train Loss: 0.693 | Train Acc: 0.50%
	 Valid Loss: 0.693 | Valid Acc: 0.49%
	 scheduled_lr : 0.00015628901896607846
Epoch: 59 | Epoch Time: 0m 39s
	 Train Loss: 0.693 | Train Acc: 0.49%
	 Valid Loss: 0.693 | Valid Acc: 0.49

[I 2022-11-29 05:56:08,782] Trial 9 finished with value: 0.6931830644607544 and parameters: {'lr': 0.0004160439645256605}. Best is trial 6 with value: 0.6724250912666321.


Epoch: 100 | Epoch Time: 0m 39s
	 Train Loss: 0.692 | Train Acc: 0.51%
	 Valid Loss: 0.693 | Valid Acc: 0.52%
	 scheduled_lr : 0.0
Best Score: 0.6724250912666321
Best trial: {'lr': 1.6400214911202294e-06}


## 6. Model Test

In [8]:
# 학습된 모델 불러오기
model.load_state_dict(torch.load(f'C:\\Users\\Bang\\JupyterProjects\\RetinaProject\\DeepLearningPytorchExample\\best_valid_model\\{args.title}.pt'))

# 모델 성능 측정
test_summary = evaluate(test_loader, model, loss_fn, metric_fn, args.device)

# write log
test_logger.add_scalar('Loss', test_summary['loss'], epoch + 1)
test_logger.add_scalar('Accuracy', test_summary['metric'], epoch + 1)

print(f'Test Loss: {test_summary["loss"]:.3f} | Test Acc: {test_summary["metric"]:.2f}%')

NameError: name 'model' is not defined

## 7. Model Prediction Print

### 1) 예측한 값 추출 + 맞춘 이미지 정보 추출 + 틀린 이미지 정보 추출

In [ ]:
correct_examples, wrong_examples = get_predictions(test_loader, model, args.device)

In [ ]:
for correct_example in correct_examples: # 확인용 코드
    print("correct_example : ", correct_example[2])
for wrong_example in wrong_examples: # 확인용 코드
    print("wrong_example : ", wrong_example[2])

In [ ]:
classes = test_dataset.classes
n_images = 5
plot_most_correct_wrong(correct_examples, wrong_examples, classes, n_images)